In [1]:

from __future__ import print_function
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from sklearn.preprocessing import StandardScaler

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 10  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'Data/fra.txt'


from sklearn.preprocessing import MinMaxScaler

n_features = 4 
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)
max_len = 0


def prepare_training(path, n_runs):
    def get_max_len(sequence_list):
        m_len = 0
        for seq in sequence_list:
            if len(seq) > m_len:
                m_len = len(seq)
        max_len = m_len
        print(max_len)
        return m_len
    
    
    def construct_matrix(sequence_list):
        max_len = get_max_len(sequence_list)
        max_len = 640
        train_matrix = np.zeros(shape=(n_runs, max_len, n_features))
        for index,run in enumerate(sequence_list):
            train_matrix[index, :len(run)] = run
        print(train_matrix.shape)
        return train_matrix
        
        
    def stadard_sequences(sequences):
        for i, seq in enumerate(sequences):
            sequences[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
            #sequences[i] = StandardScaler().fit_transform(seq)
        return sequences       
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        stand_sequences = stadard_sequences(run_list_mix)
        padded_matrix = construct_matrix(stand_sequences)
        return padded_matrix
    
    return read_sequences()
    

train_matrix = prepare_training("Mix_sequences_var_length/run^", n_runs=200) 


Using TensorFlow backend.


629
(200, 640, 4)


In [5]:

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Conv1D, Conv2DTranspose, Lambda
import keras.backend as K
filters = 50

def Conv1DTranspose(input_tensor, filters, kernel_size, strides=1, padding='same'):
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation='relu')(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 4))
    
encoder = LSTM(latent_dim, return_state=True)
x = encoder_inputs
for i in range(4):
     x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                activation='relu',
                   padding='same')(x)
     
encoder_outputs, state_h, state_c = encoder(x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

encoder = Model(encoder_inputs, encoder_states)
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 4))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
x = decoder_outputs
for i in range(2):
        x = Conv1DTranspose(input_tensor=x, filters=filters,
                            kernel_size=2, padding='same')
decoder_dense = Dense(4, activation='softmax')
decoder_outputs = decoder_dense(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()
model.compile(optimizer='rmsprop', loss='mse')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, None, 50)     450         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, None, 50)     5050        conv1d_5[0][0]                   
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, None, 50)     5050        conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_8 (

In [6]:
model.fit([train_matrix, train_matrix], train_matrix,
          batch_size=batch_size,
          epochs=75,
          validation_split=0.2)
model.save_weights("Models/Weights/LSTM_seq2seq_300_500.hdf5")


Train on 160 samples, validate on 40 samples
Epoch 1/75


 64/160 [===========>..................] - ETA: 9s - loss: 2.1253

128/160 [=======================>......] - ETA: 1s - loss: 2.1030

160/160 [==============================] - 9s 56ms/step - loss: 2.0997 - val_loss: 2.0145


Epoch 2/75


...

160/160 [==============================] - 3s 16ms/step - loss: 1.5587 - val_loss: 1.5119


Epoch 75/75


 64/160 [===========>..................] - ETA: 1s - loss: 1.5684

128/160 [=======================>......] - ETA: 0s - loss: 1.5618

160/160 [==============================] - 2s 16ms/step - loss: 1.5560 - val_loss: 1.5088


In [33]:
model.load_weights("Models/Weights/LSTM_seq2seq_1000_500.hdf5")


In [7]:
from sklearn.decomposition import PCA

encodings = encoder.predict(train_matrix)
enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[0]
print(enc_mean.shape, enc_var.shape, z_enc.shape)



def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    
    return arg




masks = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]

from mpl_toolkits.mplot3d import Axes3D  
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import numpy as np
from scipy.stats import norm


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])

    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask])

        #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    
    
    plot_pca('Sequences'+titles[i], 0)
    
    principalComponents = enc
    plot_pca('Sequences_Not_Pca'+titles[i], 0)

(200, 10) (200, 10) (200, 10)


(200, 3)
[0.52397305 0.3501349  0.10610469]


(200, 3)
[0.53138006 0.41972226 0.02193009]


(200, 3)
[0.52397305 0.3501349  0.10610469]


In [8]:

unseen_labs = pd.read_csv("Data/Boat_nominal_data/Boat_unseen_labels_mix.csv")
unseen_labs = unseen_labs.drop(columns="Unnamed: 0") 
unseen_labs = np.array(unseen_labs)
unseen_mask = [return_mask(num, np.array(unseen_labs))[:, 0] for num in range(0, 9)]

unseen_sequences_matrix = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=200)
print(unseen_sequences_matrix.shape)

unseen_encoding = encoder.predict(unseen_sequences_matrix)
print(len(unseen_encoding), unseen_encoding[0].shape)


605
(200, 640, 4)
(200, 640, 4)


2 (200, 10)


In [9]:
reconstruction = model.predict([train_matrix,train_matrix])
reconstruction_unseen = model.predict([unseen_sequences_matrix,unseen_sequences_matrix])

#RECONSTRUCTION ERROR
def get_reconstructed_matrix(input_matrix, reconstrut):
    return_matrix = np.zeros(shape=input_matrix.shape)
    for i,run in enumerate(input_matrix):
        mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
        return_matrix[i][mask_seq] = reconstrut[i][mask_seq]    
    return reconstrut


train_error = abs(train_matrix-get_reconstructed_matrix(train_matrix,reconstruction))
unseen_runs_error = abs(unseen_sequences_matrix-get_reconstructed_matrix(train_matrix,reconstruction_unseen))

train_error_avg = np.mean(train_error, axis=2)
unseen_error_avg = np.mean(unseen_runs_error, axis=2)
train_error_avg = np.mean(train_error_avg, axis=1)
unseen_error_avg = np.mean(unseen_error_avg, axis=1)
print(train_error_avg.shape, unseen_error_avg.shape)

for mask in masks:
    plt.scatter(np.linspace(1,200,200)[mask],train_error_avg[mask])
plt.title('ERROR ON TRAIN')
plt.show()

for mask in unseen_mask:
    plt.scatter(np.linspace(1,200,200)[mask],unseen_error_avg[mask])
plt.title('ERROR ON Unseen')
plt.show()

(200,) (200,)


In [13]:
def reconstruct_sequence(seq_index):
    run = train_matrix[seq_index]
    mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
    
    input = np.reshape(run, (1, run.shape[0], run.shape[1]))
    reconstr_run = model.predict([input,input])
    reconstr_run = np.reshape(reconstr_run, (run.shape[0], run.shape[1]))
    reconstr_run = reconstr_run[mask_seq]
    df = pd.DataFrame(reconstr_run, columns=["Sin", "Cosin", "Lat", "Lon"])
    plt.plot(df['Lon'], df['Lat'])
    plt.show()

for i in range(len(train_matrix)):
    reconstruct_sequence(i)
    if i > 10:
        break
    